In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/opiniones-polemicas/opiniones_polemicas.json


In [ ]:
from huggingface_hub import login
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

archivo_token = "../../tkn_huggingface_gema2b.txt"

# Leer el token desde el archivo
try:
    with open(archivo_token, "r", encoding="utf-8") as f:
        HF_TOKEN = f.read().strip()  # Elimina espacios y saltos de línea
except FileNotFoundError:
    print(f"Error: El archivo '{archivo_token}' no se encontró.")

login(token=HF_TOKEN)
model_name = "google/gemma-2b"

# Cargar tokenizer y el modelo 
tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=True)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto",  # Usa GPU automáticamente en Kaggle
    use_auth_token=True
)

print("✅ Modelo y tokenizer cargados correctamente.")


/usr/local/lib/python3.11/dist-packages/transformers/models/auto/tokenization_auto.py:898: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/33.6k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

✅ Modelo y tokenizer cargados correctamente.


In [ ]:
import json
from tqdm import tqdm

input_file = "/kaggle/input/opiniones-polemicas/opiniones_polemicas.json"
output_file = "clasificados_opiniones.json"

# Funcion de clasificacion
def classify_comment(text, strategy="zero_shot"):
    """Clasifica un comentario usando estrategias ZSL, FSL o Chain-of-thought"""
    if strategy == "zero_shot":
        prompt = f"""
        ¿El siguiente comentario contiene contenido sensible, ofensivo o inapropiado?
        Responde únicamente con 'Sí' o 'No'.
        Comentario: {text}
        """
    elif strategy == "few_shot":
        prompt = f"""
        Ejemplos:
        - Comentario: "Odio a las personas de X país." → Respuesta: Sí
        - Comentario: "Me gusta el helado de chocolate." → Respuesta: No
        ¿El siguiente comentario contiene contenido sensible, ofensivo o inapropiado?
        Comentario: {text}
        Responde únicamente con 'Sí' o 'No'.
        """
    elif strategy == "chain_of_thought":
        prompt = f"""
        Analiza el siguiente comentario paso a paso:
        1. ¿Contiene lenguaje agresivo o discriminatorio?
        2. ¿Hace referencia a temas delicados como violencia, sexo o religión de forma inadecuada?
        3. ¿Puede ser percibido como ofensivo por algún grupo?
        Finalmente, responde 'Sí' o 'No' según tu análisis.
        Comentario: {text}
        """
    else:
        raise ValueError("Estrategia no válida")

    try:
        inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512).to(model.device)

        outputs = model.generate(
            **inputs,
            max_new_tokens=20,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id,
            num_beams=1
        )

        response = tokenizer.decode(outputs[0], skip_special_tokens=True).strip().lower()
        return "Sí" if "sí" in response or "si" in response else "No"
    except Exception as e:
        print(f"[ERROR] Clasificación fallida: {e}")
        return "Error"

# Procesado principal
def process_threads():
    """Carga datos desde archivo local, clasifica comentarios y guarda resultados"""
    with open(input_file, "r", encoding="utf-8") as f:
        data = json.load(f)

    strategies = ["zero_shot", "few_shot", "chain_of_thought"]

    for thread in tqdm(data, desc="Clasificando hilos"):
        for comment in thread["comments"]:
            text = comment["comment"]
            for strategy in strategies:
                comment[f"clasificacion_{strategy}"] = classify_comment(text, strategy)

    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

    print(f"\n✅ Resultados guardados en: {output_file}")

# Mostrar 10 ejemplos 
def analyze_results():
    """Muestra ejemplos de comentarios clasificados"""
    with open(output_file, "r", encoding="utf-8") as f:
        data = json.load(f)

    examples = []
    for thread in data:
        for comment in thread["comments"]:
            if len(examples) >= 10:
                break
            examples.append(comment)

    print("\n🧠 Análisis de ejemplos clasificados:")
    for i, comment in enumerate(examples):
        print(f"\nComentario {i+1}:")
        print(f"Texto: {comment['comment'][:200]}...")
        print(f"Zero-shot: {comment['clasificacion_zero_shot']}")
        print(f"Few-shot: {comment['clasificacion_few_shot']}")
        print(f"Chain-of-thought: {comment['clasificacion_chain_of_thought']}")

process_threads()
analyze_results()


Clasificando hilos: 100%|██████████| 10/10 [13:16<00:00, 79.65s/it]


✅ Resultados guardados en: clasificados_opiniones.json

🧠 Análisis de ejemplos clasificados:

Comentario 1:
Texto: Saludos, estimado /u/Migol-16, y usuarios de r/OpinionesPolemicas.

Gracias por aportar nuevos e interesante y/o funables temas a nuestra comunidad, recuerda respetar las reglas para así tener un mejo...
Zero-shot: Sí
Few-shot: Sí
Chain-of-thought: Sí

Comentario 2:
Texto: Poco se habla que los países árabes principales (Egipto, Emiratos, Arabia) le tienen más tirria a Irán que a Israel...
Zero-shot: Sí
Few-shot: Sí
Chain-of-thought: Sí

Comentario 3:
Texto: que consideramos discurso de odio? porque si cualquier cosa que diga sobre israel va a ser tachado de antisemita pues estamos jodidos...
Zero-shot: Sí
Few-shot: Sí
Chain-of-thought: Sí

Comentario 4:
Texto: Que muchos anglofilos critican al mundo árabe de atrasado cuando los responsables son los gobiernos autoritarios y teocraricos sostenidos gracias a que detrás hay una potencia respaldandolos, por ejem...
Zero-shot: